In [1]:
import pandas as pd
import numpy as np
import re



In [5]:
df = pd.read_csv('labeled_patient_dataset_trimmed.csv', low_memory=False)

In [6]:
df.head()

,Unnamed: 0,ices_caseid,ices_age,ices_sex,ices_maristatus,ices_childstatus,ices_numchild,ices_ethnicity,ices_education,ices_workingstatus,...,ices_dosetime3,ices_addcomm6,ices_dosetime6,ices_addcomm9,ices_dosetime9,ices_addcomm12,ices_dosetime12,urine_slope,dosage_slope,label
0,831,021-0001,53.7,0,4,1,1.0,1.0,1,0,...,27.0,NaN,30.0,NaN,33.0,NaN,36.00,-2.6,0.0,0
1,832,021-0002,36.7,1,0,1,1.0,1.0,2,0,...,6.0,NaN,9.0,NaN,3.0,NaN,1.00,-8.4,-4.5,1
2,833,021-0003,61.3,1,5,1,2.0,1.0,2,0,...,9.0,NaN,12.0,NaN,15.0,NaN,2.00,-13.9,-2.0,1
3,834,021-0004,33.5,0,0,1,2.0,1.0,2,1,...,39.0,NaN,42.0,NaN,45.0,"Transferred to another provider @ March 7, 2019",46.25,-9.9,0.0,1
4,835,021-0006,38.1,0,4,1,2.0,10.0,1,0,...,3.0,NaN,6.0,NaN,9.0,NaN,12.00,-14.3,1.0,1


In [9]:
df_objects = df.select_dtypes('object')

In [8]:
df_objects

,ices_caseid,ices_b8cannabis,ices_ivtimes,ices_ivshare,ices_numsex,ices_dayspartnerconflict,ices_numpaidwork,ices_ofvehicleday,ices_mealhour,ices_osathour,...,ices_whereother,ices_legalprescriptmj,ices_salivadate,ices_salivanumber,ices_doseaddcomm2,ices_urinelabel,ices_addcomm3,ices_addcomm6,ices_addcomm9,ices_addcomm12
0,021-0001,0,NaN,.,0,NaN,0,0,2,1,...,NaN,NaN,36:00.0,POST-021-0001-S,NaN,NaN,NaN,NaN,NaN,NaN
1,021-0002,30,NaN,.,0,NaN,0,0,15,24,...,NaN,NaN,27:00.0,POST-021-0002-S,NaN,NaN,NaN,NaN,NaN,NaN
2,021-0003,0,NaN,.,0,NaN,0,0,3,27,...,NaN,NaN,11:00.0,POST-021-0003-S,NaN,NaN,NaN,NaN,NaN,NaN
3,021-0004,0,NaN,.,0,NaN,20,0,0.5,0.3,...,NaN,NaN,38:00.0,POST-021-0004-S,NaN,NaN,NaN,NaN,NaN,"Transferred to another provider @ March 7, 2019"
4,021-0006,0,NaN,.,0,NaN,0,0,72,5,...,NaN,NaN,18:00.0,POST-021-0006-S,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,050-0003,30,5,30,1,15,0,0,2,0.5,...,NaN,NaN,32:00.0,POST-050-0003-S,NaN,NaN,Failed treatment @ 2020-01-23 came back before...,NaN,Restarts within one month cutoff,NaN
1730,050-0007,30,NaN,.,1,25,0,0,48,24,...,NaN,NaN,46:00.0,POST-050-0007-S,NaN,NaN,NaN,NaN,NaN,NaN
1731,050-0008,.,NaN,.,0,NaN,.,0,NaN,0,...,NaN,NaN,28:00.0,POST-050-0008-S,NaN,NaN,NaN,NaN,NaN,NaN
1732,050-0010,30,2,0,0,15,0,0,15,24,...,NaN,NaN,NaN,NaN,SAMPLE DRIED UP NEED NEW SAMPLE,NaN,NaN,NaN,NaN,NaN


In [10]:
for col in df_objects.columns:
    df_objects.loc[:,col] = df_objects[col].replace('.',0).copy()

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [11]:
df_objects

,ices_caseid,ices_b8cannabis,ices_ivtimes,ices_ivshare,ices_numsex,ices_dayspartnerconflict,ices_numpaidwork,ices_ofvehicleday,ices_mealhour,ices_osathour,...,ices_whereother,ices_legalprescriptmj,ices_salivadate,ices_salivanumber,ices_doseaddcomm2,ices_urinelabel,ices_addcomm3,ices_addcomm6,ices_addcomm9,ices_addcomm12
0,021-0001,0,NaN,0,0,NaN,0,0,2,1,...,NaN,NaN,36:00.0,POST-021-0001-S,NaN,NaN,NaN,NaN,NaN,NaN
1,021-0002,30,NaN,0,0,NaN,0,0,15,24,...,NaN,NaN,27:00.0,POST-021-0002-S,NaN,NaN,NaN,NaN,NaN,NaN
2,021-0003,0,NaN,0,0,NaN,0,0,3,27,...,NaN,NaN,11:00.0,POST-021-0003-S,NaN,NaN,NaN,NaN,NaN,NaN
3,021-0004,0,NaN,0,0,NaN,20,0,0.5,0.3,...,NaN,NaN,38:00.0,POST-021-0004-S,NaN,NaN,NaN,NaN,NaN,"Transferred to another provider @ March 7, 2019"
4,021-0006,0,NaN,0,0,NaN,0,0,72,5,...,NaN,NaN,18:00.0,POST-021-0006-S,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,050-0003,30,5,30,1,15,0,0,2,0.5,...,NaN,NaN,32:00.0,POST-050-0003-S,NaN,NaN,Failed treatment @ 2020-01-23 came back before...,NaN,Restarts within one month cutoff,NaN
1730,050-0007,30,NaN,0,1,25,0,0,48,24,...,NaN,NaN,46:00.0,POST-050-0007-S,NaN,NaN,NaN,NaN,NaN,NaN
1731,050-0008,0,NaN,0,0,NaN,0,0,NaN,0,...,NaN,NaN,28:00.0,POST-050-0008-S,NaN,NaN,NaN,NaN,NaN,NaN
1732,050-0010,30,2,0,0,15,0,0,15,24,...,NaN,NaN,NaN,NaN,SAMPLE DRIED UP NEED NEW SAMPLE,NaN,NaN,NaN,NaN,NaN


In [22]:
df_objects['ices_osathour'].value_counts()

0.5       448
24        307
1          93
25         62
26         61
         ... 
30.5        1
0.8         1
00-Jan      1
44          1
53          1
Name: ices_osathour, Length: 107, dtype: int64

In [20]:
sum(pd.to_numeric(df_objects['ices_osathour'], errors='coerce').value_counts().values)

1733

In [15]:
sum(df_objects['ices_numsex'].value_counts().values)/1734

1.0

In [23]:
col_nulls = {}
for col in df_objects.columns:
    if sum(df_objects[col].value_counts().values)/1734 > 0.6:
        col_nulls[col] = sum(df_objects[col].value_counts().values)/1734

1) convert all the rows to numeric
2) check how many non-null entries in each column 
3) if the percent is "decent" then we might want to keep the column and manually set things to zero

In [24]:
col_nulls

{'ices_caseid': 1.0,
 'ices_b8cannabis': 1.0,
 'ices_ivshare': 1.0,
 'ices_numsex': 1.0,
 'ices_numpaidwork': 1.0,
 'ices_ofvehicleday': 0.9994232987312572,
 'ices_mealhour': 0.9971164936562861,
 'ices_osathour': 1.0,
 'ices_dose3': 1.0,
 'ices_urinestart3': 1.0,
 'ices_urineend3': 0.9994232987312572,
 'ices_dose6': 1.0,
 'ices_length6': 1.0,
 'ices_urinestart6': 0.9988465974625144,
 'ices_urineend6': 0.9994232987312572,
 'ices_dose9': 1.0,
 'ices_urinestart9': 0.9994232987312572,
 'ices_urineend9': 1.0,
 'ices_dose12': 1.0,
 'ices_urinestart12': 1.0,
 'ices_urineend12': 1.0,
 'ices_postgenoa': 1.0,
 'ices_opioiddependence': 0.9994232987312572,
 'ices_goalsosat': 0.9994232987312572,
 'ices_medhistory': 0.9803921568627451,
 'ices_currentmeds': 0.9492502883506344,
 'ices_mentalhealthmeds': 0.9619377162629758,
 'ices_mjlegaluse': 0.9607843137254902,
 'ices_regularuse': 0.961361014994233,
 'ices_salivadate': 0.9988465974625144,
 'ices_salivanumber': 0.9994232987312572}

In [27]:
for col in df_objects.columns:
    if sum(pd.to_numeric(df_objects[col], errors='coerce').isnull()) > 800:
        df_objects.drop(col,axis=1,inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
num_nulls = {}
for col in df_objects.columns:
    num_nulls[col] = sum(pd.to_numeric(df_objects[col], errors='coerce').isnull())
    


In [29]:
sorted(num_nulls.items(), key=lambda x: x[1], reverse=True)

[('ices_dayspartnerconflict', 722),
 ('ices_regularuse', 201),
 ('ices_mealhour', 31),
 ('ices_ivshare', 2),
 ('ices_ofvehicleday', 1),
 ('ices_osathour', 1),
 ('ices_length6', 1),
 ('ices_b8cannabis', 0),
 ('ices_numsex', 0),
 ('ices_numpaidwork', 0)]

# Regular Expression Exp

In [31]:
col = 'ices_mentalhealthmeds'

df[col]

0                            No
1                          None
2              valium , effexor
3                            No
4         Citraline, Olanzapine
                 ...           
1729                         No
1730    Amitriptyline, cipralex
1731                       none
1732                          0
1733                         no
Name: ices_mentalhealthmeds, Length: 1734, dtype: object

If record has 'no' or 'none' or any variation of this, then they do not take meds, therefore 0. If they do have a drug we are hoping the drug doesn't have 'no' in it.

In [61]:
type(df['ices_mentalhealthmeds'].iloc[1732])

str

In [46]:
no = 'no'

no, noo, none, NO, no9812398123981iuehfjka

In [71]:
string = 'yes'

In [72]:
def has_no(col):
    try:
        if len(re.findall(r'^[\(no\)0]*', col.lower())) == 1:
            return 0
        else:
            return 1
    except:
        return 0
        


In [73]:
has_no(string)

0

In [57]:
df['ices_mentalhealthmeds'].apply(has_no)

0       0
1       0
2       1
3       0
4       1
       ..
1729    0
1730    1
1731    0
1732    1
1733    0
Name: ices_mentalhealthmeds, Length: 1734, dtype: int64

In [83]:
for col in df_objects:
    df.loc[:,col] = df_objects.loc[:,col]

In [51]:
no = 'yes'

In [71]:
def has_no(col):
    try:
        if len(re.findall(r'^no*',col.lower())):
            return 0
        return 1
    except:
        return 0

In [57]:
has_no(no)

1

In [72]:
df_objects.loc[:,'ices_mentalhealthmeds_bin'] = df_objects['ices_mentalhealthmeds'].apply(has_no).copy()

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [76]:
df_objects.ices_mentalhealthmeds_bin.value_counts()

0    969
1    765
Name: ices_mentalhealthmeds_bin, dtype: int64

In [9]:
sorted(num_nulls.items(), key=lambda x: x[1], reverse=True)

[('ices_caseid', 1734),
 ('ices_addcomm1', 1734),
 ('ices_othereth', 1734),
 ('ices_lastworking', 1734),
 ('ices_howother', 1734),
 ('ices_datelastdose', 1734),
 ('ices_otherprescript', 1734),
 ('ices_b9name', 1734),
 ('ices_urinestartbase', 1734),
 ('ices_urineendbase', 1734),
 ('ices_dose3', 1734),
 ('ices_urinestart3', 1734),
 ('ices_urineend3', 1734),
 ('ices_dose6', 1734),
 ('ices_urinestart6', 1734),
 ('ices_urineend6', 1734),
 ('ices_dose9', 1734),
 ('ices_urinestart9', 1734),
 ('ices_urineend9', 1734),
 ('ices_dose12', 1734),
 ('ices_urinestart12', 1734),
 ('ices_urineend12', 1734),
 ('ices_postgenoa', 1734),
 ('ices_ethmixed', 1734),
 ('ices_occupation', 1734),
 ('ices_nameofscript', 1734),
 ('ices_nocontinuescript', 1734),
 ('ices_opioiddependence', 1734),
 ('ices_howotherdepend', 1734),
 ('Ices_currentopoth', 1734),
 ('ices_edreasonother', 1734),
 ('ices_obtainnal', 1734),
 ('ices_subvape', 1734),
 ('ices_otherosat', 1734),
 ('ices_goalsosat', 1734),
 ('ices_anyotherosat', 1

1) try pd.to_numerics(error='coerce') --> converts any non-numeric entry to a NULL
2) count the nulls
3) if the nulls are really small then dont drop the column



* ices_daysfriendsconflict : convert to 0
* ices_regularuse : convert to 0
* ices_totalyearsused : split string, if num take num else 0


In [10]:
col = 'ices_weight'

In [14]:
sum(pd.to_numeric(df_objects[col], errors='coerce').isna())

14

In [20]:
df.loc[pd.to_numeric(df[col], errors='coerce').isnull(),col] = 0

In [21]:
df.loc[pd.to_numeric(df[col], errors='coerce').isnull(),col]

Series([], Name: ices_weight, dtype: object)

In [16]:
df[col]

0        68
1        84
2        52
3        66
4        61
       ... 
1729     99
1730     66
1731     66
1732    NaN
1733     54
Name: ices_weight, Length: 1734, dtype: object

In [17]:
pd.to_numeric(df[col], errors='coerce').isnull()


0       False
1       False
2       False
3       False
4       False
        ...  
1729    False
1730    False
1731    False
1732     True
1733    False
Name: ices_weight, Length: 1734, dtype: bool

In [ ]:
def print_nonnumeric(col):
    

In [ ]:
def convert_to_zero(col):
    if col == 0 & col == '.':
        

In [85]:
df.to_csv('patient_data_trimmed_numericed.csv')

In [87]:
df_obj

NameError: name 'df_obj' is not defined